# Example: Time delay neural network


Process outline
1. Prepare time delay features
2. Spatial aggregation
   - Area weighted sum for precipitation features
   - Area mean for all other features
3. Define the model
4. Train the model
   - Reshape the input dataset to the model specific `X` and `y` arrays
   - Actually train on `X` and `y`

In [1]:
import numpy as np
import datetime as dt
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import dask
dask.config.set(scheduler='threads')
import xarray as xr

## Loading the data
Sample dataset contained in the git repository.

As you are reading these lines, you opened the notebook in the `./docs/` folder of the main repository directory. To access the sample dataset that was delivered to you with the code, step outside the current directory (`../`) and enter the `data/` folder. The names of the sample datasets are `smallsampledata-era5.nc` and `smallsampledata-glofas.nc`, both in netCDF format, a user-friendly format that keeps the file size low and stores meta-data within the file.

We use `xarray` to access the files, as it provides us with a very powerful interface to work with the data. Let's open the files and see what's in there:

In [2]:
era5 = xr.open_dataset('../data/smallsampledata-era5.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/raid/home/srvx7/lehre/users/a1254888/ipython/ml_flood/notebooks/data/smallsampledata-era5.nc'

In [ ]:
glofas = xr.open_dataset('../data/smallsampledata-glofas.nc')

Its datatype is `xarray.Dataset`, containing a spatio-temporal subset of 5 variables that we selected from the original ERA5 dataset. It also contains one derived variable (`rtp_500-850`) that is used in Meteorology quite a bit because it is proportional to mass-weighted mean temperature of the air, in this case between 850 and 500 hPa. It could be an additional predictor to classify the weather regime.

In [ ]:
era5

To access a variable's description, select one like this:

In [ ]:
era5['cp']

### Selecting useful predictor variables
The selection has already been done by us when preparing the small sample datasets for you, so you can use them straight away. What is still necessary, is clustering and reshaping the input data dimension, because the dimensionality of the raw input data would be too high: 

To give a rough estimate, imagine using all GloFAS and ERA5 gridpoints of the upstream area of one point. For 1.5x1.5  degree (lat,lon), ERA5 provides 6 x 6 and GloFAS 15 x 15 gridpoints. To take the time dimension into account we'd need, say, 10 days of discharge and 180 days of precipitation. 

Summing up, this makes the raw input dimensionality 15 x 15 x 10 and 6 x 6 x 180, in total ~8700 features, where most of the predictors won't vary that much from one gridpoint to another in the upstream area (large scale temperature, total-column water vapor).

To work around that, we need to 
  1. spatially aggregate  and 
  2. temporally aggregate the data (reduce dimensionality). 
  
Both will be done in sections below, but first we need to define which point we will be making forecasts for.

### Selecting the predictand
The target variable shall be the discharge at the point where the Danube river leaves the data domain. This is given by the point within the catchment where the discharge is the highest, so we first filter by the catchment basin shapefile (Worldbank dataset "Global River Basins")

To select the basin from the shapefile, we use a function defined in `./aux/utils.py`. The interested user may have a look there how it is done, but it would obstruct the clarity in this notebook. To import the function it needs to be present in the current processes path. We do that by adding the parent/main directory to `sys.path`. In this way we can import some function in `./aux/utils_flowmodel.py` by `from aux.utils_flowmodel import somefunction`.

In [ ]:
import sys
sys.path.append("..")
from python.aux.utils_flowmodel import get_mask_of_basin

In [ ]:
danube_catchment = get_mask_of_basin(glofas['dis'].isel(time=0))
dis = glofas['dis'].where(danube_catchment)
dis_mean = dis.mean('time')

Now we look up the coordinates of the maximum discharge point. We see that the point of interest is at 48.35 degree latitude and 13.95 degree longitude.

In [ ]:
maximum = dis.where(dis==dis.max(), drop=True)
lat, lon = float(maximum.latitude), float(maximum.longitude)
maximum

To double-check, plot a circle around the point of interest. Indeed, its the point we looked for.

In [ ]:
dis.mean('time').plot(cmap='gist_ncar')
plt.gca().plot(lon, lat, color='cyan', marker='o', 
                     markersize=20, mew=4, markerfacecolor='none')

## 1. Time aggregated predictors
Delayed effect

=> delayed impact, time-shifted precipitation variables

The goal shall be to aggregate over an increasing number of days as we iterate back in time.
What is shifting?

Initial setup `X`:

In [ ]:
X = era5[['lsp', 'cp']]

In [ ]:
from python.aux.utils_flowmodel import shift_and_aggregate
from python.aux.utils import calc_area, nandot

In [ ]:
for var in ['lsp', 'cp']:
    for i in range(1,6):
        newvar = var+'-'+str(i)
        X[newvar] = X[var].shift(time=i)  # previous precip as current day variable
        
    for i in range(1,14):
        newvar = var+'+'+str(i)
        X[newvar] = X[var].shift(time=-i) # future precip as current day variable
        

#### Finally, put the time aggregated predictors together
Apply `shift_and_aggregate` before interpolating the fields to avoid MemoryErrors.


In [ ]:
X['lsp-5-11'] = shift_and_aggregate(X['lsp'], shift=5, aggregate=7)
X['lsp-12-25'] = shift_and_aggregate(X['lsp'], shift=12, aggregate=14)
X['lsp-26-55'] = shift_and_aggregate(X['lsp'], shift=26, aggregate=30)
X['lsp-56-180'] = shift_and_aggregate(X['lsp'], shift=56, aggregate=125)

## 2. Spatially aggregated predictors
We can either take the total sum of precipitation that falls throughout the basin, or we can aggregate it by time to the point of interest (POI). As we can see from the discharge plot above, points with less mean discharge are further away from the POI compared to gridpoints with a lighter color. This is the motivation to cluster the precipitation points by the mean discharge of the gridpoint. So we need discharge bins, according to which the precipitation at these gridpoints is grouped together to form one feature/predictor. 

For an equal number of points per bin, we can have a look on the percentiles of the discharge distribution. As a first guess let's take four clusters so that the percentiles 0.25, 0.5 and 0.75 are our threshold values between the four clusters: 

In [ ]:
for q in [0.25, .5, .75]:
    print('percentile', q, ': ', round(float(dis_mean.quantile(q)),3), 'm^3/s')

Points with less than 0.8 $m^3/s$ discharge are the first cluster, points with 0.8-2.5 $m^3/s$ another cluster and so on. Finally we can add a visual to this, the cumulative distribution of discharges.

We see that 80 percent of all gridpoints exhibit a mean discharge of less than 20 m^3/s, and that there is a sharp change in slope between 5 and 10 m^3/s discharge.

### Create the clusters-masks

In [ ]:
from python.aux.utils_flowmodel import cluster_by_discharge
from python.aux.utils import calc_area, nandot

bin_edges = [0, 0.8, 2.5, 10.25, 10000]
cluster = cluster_by_discharge(dis_mean, bin_edges)

To create an image with all clusters in one image, 
we will create a label-array containing a number between 0 and 3 for every gridpoint 
that classifies each gridpoint belonging to one of the four categories. 

In numpy you would probably use the numpy boolean masking operation:
```
for i in range(len(clusters)):
    image[clustering[c]] = i  ```
but you would notice an `IndexError`, as xarray does not support 2-dimensional boolean indexing. So we have to formulate it this way: `image = image.where(~clustering[c], i)`, where not 'this cluster', do nothing, else overwrite with the cluster index

In [ ]:
image = dis_mean*0.
image.name = 'spatial feature cluster'
for i, c in enumerate(cluster):
    image = image.where(~cluster[c], i)
    
image.plot(cmap = mpl.colors.ListedColormap(['grey', 'orange', 'blue', 'darkblue']))

These are the gridpoints that are aggregated together to form one precipitation feature for the forecast model. For precipitation that is older than a few days, only the first cluster is of interest. Precipitation that occured on the other gridpoints is already transported outside of the domain for sure.


In [ ]:
cluster = cluster.to_array('clusterId')
cluster.coords 

In [ ]:
from python.aux.utils_floodmodel import aggregate_clustersum

In [ ]:
Xagg = aggregate_clustersum(X, cluster, 'clusterId')

In [ ]:
# drop these predictors
for v in Xagg:
    if 'cluster0' in v:
        for vn in ['lsp-5-11', 'lsp-12-25', 'lsp-26-55', 'lsp-56-180']:
            if vn in v:
                Xagg = Xagg.drop(v)
                break

In [ ]:
# drop these predictors (predictand time)
for v in Xagg:
    for vn in ['lsp_cluster', 'cp_cluster']:
        if v.startswith(vn):
            Xagg = Xagg.drop(v)
            break

In [ ]:
if False:  # alternative: aggregating over space by taking the mean
    Xagg = X.mean(['latitude', 'longitude'])
Xagg

In [ ]:
y = glofas['dis'].interp(latitude=lat, longitude=lon)

var = y.name
y = y.to_dataset()
for i in range(1,14):
    newvar = var+'+'+str(i)
    y[newvar] = y[var].shift(time=-i) # future precip as current day variable
y = y.to_array(dim='forecast_day')
y.coords['forecast_day'] = range(1,len(y.forecast_day)+1)

In [ ]:
y

In [ ]:
from python.aux.utils_flowmodel import reshape_multiday_predictand

In [ ]:
Xt = Xagg.to_array(dim='var_dimension')
Xt

In [ ]:
Xt

In [ ]:
Xda, yda = reshape_multiday_predictand(Xagg, y)
Xda

In [ ]:
period_train = dict(time=slice(None, '1990'))
period_valid = dict(time=slice('1991', '1993'))
period_test = dict(time=slice('1994', '1995'))

In [ ]:
X_train, y_train = Xda.loc[period_train], yda.loc[period_train]
X_valid, y_valid = Xda.loc[period_valid], yda.loc[period_valid]
X_test, y_test = Xda.loc[period_test], yda.loc[period_test]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
from sklearn_xarray import wrap
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import keras
from keras.layers.core import Dropout
from keras.constraints import MinMaxNorm, nonneg

def add_time(vector, time, name=None):
    """Converts arrays to xarrays with a time coordinate."""
    return xr.DataArray(vector, dims=('time'), coords={'time': time}, name=name)

def ensure_nparray(a):
    if isinstance(a, xr.DataArray):
        a = a.values
    return a

class DenseNN(object):
    def __init__(self, **kwargs):
        self.xscaler = StandardScaler()
        self.yscaler = StandardScaler()
        model = keras.models.Sequential()
        self.cfg = kwargs
        hidden_nodes = self.cfg.get('hidden_nodes')
        
        for n in hidden_nodes:
            model.add(keras.layers.Dense(n, activation='elu',)) 
                                         #kernel_constraint=MinMaxNorm(min_value=0.0, max_value=1.0, rate=1.0, axis=0)))
            model.add(keras.layers.BatchNormalization())
            model.add(Dropout(self.cfg.get('dropout', None)))
        model.add(keras.layers.Dense(kwargs.get('output_dim', 1), 
                                     activation='linear'))

        opt = keras.optimizers.Adadelta() #lr=1, rho=0.95, epsilon=0.5, decay=0.0)
        #opt = keras.optimizers.RMSprop()
        #opt = keras.optimizers.SGD()
        #opt = keras.optimizers.Adam(lr=0.001, epsilon=None, amsgrad=False)

        model.compile(loss=self.cfg.get('loss'), optimizer=opt)
        self.model = model

        self.callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                            min_delta=1e-2, patience=100, verbose=0, mode='auto',
                            baseline=None, restore_best_weights=True),]

    def predict(self, Xda, name=None):
        #X = ensure_nparray(Xda)
        X = self.xscaler.transform(Xda.values)
        y = self.model.predict(X).squeeze()
        y = self.yscaler.inverse_transform(y)
        return add_time(y, Xda.time, name=name)

    def fit(self, X_train, y_train, X_valid, y_valid, **kwargs):
        # xarray in !
        X_train = self.xscaler.fit_transform(X_train.values)
        y_train = self.yscaler.fit_transform(y_train.values.reshape(-1, self.cfg.get('output_dim', 1)))
        
        X_valid = self.xscaler.transform(X_valid.values)
        y_valid = self.yscaler.transform(y_valid.values.reshape(-1, self.cfg.get('output_dim', 1)))
        
        return self.model.fit(X_train, y_train,
                              validation_data=(X_valid, y_valid), 
                              epochs=self.cfg.get('epochs', 1000),
                              batch_size=self.cfg.get('batch_size'),
                              callbacks=self.callbacks,
                              verbose=0, **kwargs)

In [ ]:
m = DenseNN(hidden_nodes=(32,16,8,4,),
            dropout=0.25,
            output_dim=14,
            epochs=1000,
            batch_size=180,
            loss='mse')

In [ ]:
hist = m.fit(X_train, y_train, X_valid, y_valid)

In [ ]:
m.model.summary()

In [ ]:
def add_time(vector, time, name=None):
    """Converts arrays to xarrays with a time coordinate."""
    return xr.DataArray(vector, dims=('time'), coords={'time': time}, name=name)

def add_time_to_sequence_output(array, time, name=None):
    """Add time to output of models giving multi-valued output."""
    #coords = dict()
    #for i in range(array.shape[1]):
    #    coords['time+'+str(i)] = time.shift(time=i)
    #print(coords)
    init_time = pd.to_datetime(time.values)-dt.timedelta(hours=1)
    fxh = range(1,array.shape[1]+1)
    #print(valid_time.shape)
    return xr.DataArray(array, dims=('init_time', 'fxh'), 
                        coords=dict(init_time=('init_time', init_time),
                                    fxh=('fxh', fxh), 
                                    name=name))

In [ ]:
h = hist.model.history

# Plot training & validation loss value
fig, ax = plt.subplots()
ax.plot(h.history['loss'], label='loss')
ax.plot(h.history['val_loss'], label='val_loss')
plt.title('Model loss')
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
plt.legend() #['Train', 'Test'], loc='upper left')
ax.set_yscale('log')
#fig.savefig(f_hist); plt.close('all')

In [ ]:
out = m.predict(X_test)
out.to_netcdf('../../models/time-delay-NN.nc')
print(out.mean())

In [ ]:
timeslice = slice('1984', '1985')
fix, ax = plt.subplots(figsize=(15,5))
y_valid.plot(label='y')
out.plot(label='y_predict')
ax.legend()

In [ ]:
fix, ax = plt.subplots(figsize=(15,5))
y_test.plot(label='y')
out.plot(label='y_predict')
ax.legend()
plt.title('test period: '+str(period_test))